# Tutorial: the basics of NGMT

**Author:** Julius Welzel
**Last update:** Fri 22 Mar 2024

## Learning objectives
By the end of this tutorial:
- you can load data and channel information into an `NGMTRecording` object
- you can add Recording specific information to the `NGMTRecording` object
- you are educated about the relationship between a `NGMTRecording` object and the [BIDS standard](https://bids-specification.readthedocs.io/en/stable/).

## Imports

We start by importing some Python libraries. You should be familiar with most of them, and we will not discuss them here.

In [1]:
from pathlib import Path
import pandas as pd
import os

from ngmt.utils.ngmt_dataclass import NGMTRecording # Import the NGMTRecording class
from ngmt.modules.gsd import ParaschivIonescuGaitSequenceDetection # Import the Gait Sequence Detection module

## Read the raw data and metadata into python

Let us consider a single recording, a accelerometer located at the lower back.
The sensor only provided the raw data in a csv file without any metadata. We only know the metadata from the sensor manual and from setting up the sensor.
Depending on the sensor, the metadata might be stored in a separate file or in the same file as the raw data as a header.

To import data from your own devices, check if NGMT already has a reader for your device. If not, you can write your own reader and contribute it to the NGMT project.

In [2]:
# Set the filepath
file_path = Path(os.getcwd()).parent.joinpath("examples","data","exAccelDataMobilise.csv")

# read the csv data
acc_data = pd.read_csv(file_path)

# specifiy the sampling rate
fs = 100 # Hz

We have loaded the data for one tracking systems, `LB_ACCEL`. We know we have three channels: `x`, `y`, and `z`. We also know that the data is sampled at 100 Hz.

Let's check if the data is loaded correctly. We know we have a good 15 minutes of data with three channels representing acceleration.


In [3]:
n_channels = acc_data.shape[1]
n_samples = acc_data.shape[0]

# check how many minutes of data we have
n_minutes = n_samples/fs/60
print(f"Minutes of data: {n_minutes:.2f}")


Minutes of data: 18.33


Now we can load the data into an `NGMTRecording` object.
For this we have to construct the channel information oursleves. We know that the data is sampled at 100 Hz and we have three channels: `x`, `y`, and `z`.
For more information in the channel information, see the [BIDS standard](https://bids-specification.readthedocs.io/en/stable/modality-specific-files/motion.html#channels-description-_channelstsv).

**CAVE**:
If the naming of the channels df is not following the BIDS standard, you will receive an error.

In [4]:
# construct the channel information

tracked_point = "lowerBack"

n_channels = acc_data.shape[1] # second dimension of the data represents the number of channels

col_names = [
    f"{tracked_point}_{s}_{x}"
    for s in ["ACCEL"]
    for x in ["x", "y", "z"]
]

# Make the channel dictionary following the BIDS naming conventions
channels_dict = {
    "name": col_names,
    "component": ["x", "y", "z"] * (n_channels // 3),
    "type": ["ACCEL"] * (n_channels),
    "tracked_point": [tracked_point] * n_channels,
    "units": ["m/s^2"] * n_channels,
    "sampling_frequency": [fs] * n_channels,
}

recording =  NGMTRecording(
    data={"imu": acc_data}, channels={"lb_imu": pd.DataFrame(channels_dict)}
)

Ahh, perfect. Let's first inspect the channels in the dataclass.

In [5]:
recording.channels

{'lb_imu':                 name component   type tracked_point  units  sampling_frequency
 0  lowerBack_ACCEL_x         x  ACCEL     lowerBack  m/s^2                 100
 1  lowerBack_ACCEL_y         y  ACCEL     lowerBack  m/s^2                 100
 2  lowerBack_ACCEL_z         z  ACCEL     lowerBack  m/s^2                 100}

Now we can run some of our modules on this data. For example, we find gait sequences in our data.

In [9]:
recording.data["imu"]

,0,1,2
0,0.969647,-0.033464,-0.001778
1,0.970406,-0.036270,-0.000252
2,0.973402,-0.040885,0.000498
3,0.972774,-0.038731,0.005013
4,0.974063,-0.036836,-0.001207
...,...,...,...
109995,0.970364,0.024569,0.095281
109996,0.968783,0.023735,0.095931
109997,0.969204,0.027664,0.098075
109998,0.968436,0.029833,0.096733


In [10]:
# Create an instance of the ParaschivIonescuGaitSequenceDetection class
gsd = ParaschivIonescuGaitSequenceDetection(target_sampling_freq_Hz=40)

# Call the gait sequence detection using gsd.detect
gsd = gsd.detect(
    data=recording.data["imu"], sampling_freq_Hz=fs, plot_results=False, dt_data=None
)

# Gait sequences are stored in gait_sequences_ attribute of gsd
gait_sequences = gsd.gait_sequences_

# Add events to the recording as a dictionary including tracking system and events
gait_sequence_events = gait_sequences
recording.add_events(tracking_system=tracked_point, new_events=gait_sequence_events)

# Show events and their corresponding information
print(recording.events)

20 gait sequence(s) detected.
{'lowerBack':        onset  duration     event_type tracking_systems tracked_points
0     90.175    10.400  gait sequence               SU      LowerBack
1    106.075     5.600  gait sequence               SU      LowerBack
2    121.750     4.250  gait sequence               SU      LowerBack
3    141.275     5.525  gait sequence               SU      LowerBack
4    195.025     7.100  gait sequence               SU      LowerBack
5    207.850    12.325  gait sequence               SU      LowerBack
6    256.925     5.900  gait sequence               SU      LowerBack
7    291.175    16.650  gait sequence               SU      LowerBack
8    319.450     7.100  gait sequence               SU      LowerBack
9    360.350    32.375  gait sequence               SU      LowerBack
10   408.125    58.650  gait sequence               SU      LowerBack
11   470.975     4.275  gait sequence               SU      LowerBack
12   479.150     3.600  gait sequence         